# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2621f41178-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series HIMYM explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should

1. Identify the BGP for tv series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for tv series

In [6]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/

In [2]:
# 'The office' is istance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q23831 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


Final query for this task

In [3]:
# write your final query

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q23831 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [4]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
   FILTER regex(?name, "season", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
1


In [5]:
# 'television series' details
queryString = """
SELECT DISTINCT ?p ?name ?obj ?o_name
WHERE {
   # bind something
   wd:Q5398426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1113'), ('o_name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15416'), ('o_name', 'television program')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('o_name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('o_name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1476'), ('o_name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('name', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q131758'), ('o_name', 'Desperate Housewives')]
[('p

In [7]:
# find season in the objects related to The Office
queryString = """
SELECT DISTINCT ?p ?name ?obj ?o_name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?o_name.
   
   FILTER regex(?o_name, "season", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3468906'), ('o_name', 'The Office, season 4')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3730261'), ('o_name', 'The Office, season 6')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3468601'), ('o_name', 'The Office, season 2')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3730253'), ('o_name', 'The Office, season 9')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3465812'), ('o_name', 'The Office, season 1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3730255'), ('o_name', 'The Office, season 8')]
[('p', 'http://www.w

In [8]:
# 'The Office, season 4' details
queryString = """
SELECT DISTINCT ?p ?name ?obj ?o_name
WHERE {
   # bind something
   wd:Q3468906 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('o_name', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('o_name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series'), ('obj', 'http://www.wikidata.org/entity/Q23831'), ('o_name', 'The Office')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q3464665'), ('o_name', 'television series season')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q16744428'), ('o_name', 'Job Fair')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q50379716'), ('o_name', 'Goodbye, Toby (part 1)')]
[('p

In [4]:
# 'television series season' details
queryString = """
SELECT DISTINCT ?p ?name ?obj ?o_name
WHERE {
   # bind something
   wd:Q3464665 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1113'), ('o_name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q70990126'), ('o_name', 'creative work season')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15416'), ('o_name', 'television program')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('o_name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('o_name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q21191270'), ('o_name', 'televisio

In [6]:
# General tv series seasons and # of episodes

queryString = """
SELECT DISTINCT ?s ?name ?ne
WHERE {
   # bind something
   ?ts wdt:P31 wd:Q5398426 ;
       wdt:P527 ?s .
    ?s wdt:P1113 ?ne .
       
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3061434'), ('name', 'Law & Order: Special Victims Unit, season 4'), ('ne', '25')]
[('s', 'http://www.wikidata.org/entity/Q15726792'), ('name', 'Face Off, season 6'), ('ne', '15')]
[('s', 'http://www.wikidata.org/entity/Q15844444'), ('name', 'Schloss Einstein, season 2'), ('ne', '40')]
[('s', 'http://www.wikidata.org/entity/Q463125'), ('name', 'ER, season 3'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q463131'), ('name', 'ER, season 6'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q463133'), ('name', 'ER, season 7'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q463138'), ('name', 'ER, season 5'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q463144'), ('name', 'ER, season 8'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q16023761'), ('name', 'Spiral, season 3'), ('ne', '12')]
[('s', 'http://www.wikidata.org/entity/Q463092'), ('name', 'ER, season 10'), ('ne', '22')]
[('s', 'http://www.wikidata.o

Final query for this task

In [3]:
# write your final query

queryString = """
SELECT DISTINCT ?s ?name ?ne
WHERE {
   # bind something
    wd:Q147235 wdt:P527 ?s .
    ?s wdt:P1113 ?ne .
       
   # get the label
   ?s sc:name ?name.
}
Order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2715578'), ('name', 'How I Met Your Mother, season 1'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q3468515'), ('name', 'How I Met Your Mother, season 2'), ('ne', '22')]
[('s', 'http://www.wikidata.org/entity/Q2555117'), ('name', 'How I Met Your Mother, season 3'), ('ne', '20')]
[('s', 'http://www.wikidata.org/entity/Q2567330'), ('name', 'How I Met Your Mother, season 4'), ('ne', '24')]
[('s', 'http://www.wikidata.org/entity/Q582332'), ('name', 'How I Met Your Mother, season 5'), ('ne', '24')]
[('s', 'http://www.wikidata.org/entity/Q2438066'), ('name', 'How I Met Your Mother, season 6'), ('ne', '24')]
[('s', 'http://www.wikidata.org/entity/Q2472427'), ('name', 'How I Met Your Mother, season 7'), ('ne', '24')]
[('s', 'http://www.wikidata.org/entity/Q338715'), ('name', 'How I Met Your Mother, season 8'), ('ne', '24')]
[('s', 'http://www.wikidata.org/entity/Q13567027'), ('name', 'How I Met Your Mother, season 9'), ('ne', '24')]
9


## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [7]:
# Things related to "HIMYM"
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q147235 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5398810'), ('name', 'Template:How I Met Your Mother'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('p_name', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q2332356'), ('name', 'How I Wet Your Mother'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('p_name', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q20385035'), ('name', 'Ahla Ayam'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('p_name', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q4208862'), ('name', 'How I Met Your Mother'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('p_name', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q2347474'), ('name', 'Robin Scherbatsky'), ('p', 'http://www.wikidata.org/prop/direct/P1441'), ('p_name', 'present in work')]
[('s', 'http://www.wikidata.org/entity/Q50383228'), ('name', 'Don Frank'), ('p', 'http://www.wikidata.org/prop/direct/P1441'), ('p_name', 'present in work')]
[('s', 'http:

In [8]:
# Predicates related to "HIMYM"
queryString = """
SELECT DISTINCT ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q147235 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('p_name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('p_name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('p_name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('p_name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('p_name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('p_name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('p_name', 'notable work')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('p_name', 'category combines topics')]
8


In [10]:
# "Barney Stinson" details
queryString = """
SELECT DISTINCT ?p ?p_name ?s ?name
WHERE {
   # bind something
   wd:Q838512 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('p_name', 'employer'), ('s', 'http://www.wikidata.org/entity/Q50384006'), ('name', 'Goliath National Bank')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('p_name', 'languages spoken, written or signed'), ('s', 'http://www.wikidata.org/entity/Q1860'), ('name', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('p_name', 'present in work'), ('s', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('p_name', 'creator'), ('s', 'http://www.wikidata.org/entity/Q2656047'), ('name', 'Carter Bays')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('p_name', 'creator'), ('s', 'http://www.wikidata.org/entity/Q3002075'), ('name', 'Craig Thomas')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('p_name', 'performer'), ('s', 'http://www.wikidata.org/entity/Q485310'), ('name', 'Neil Patrick Harris')]
[('p', 'http://www.wikidata.or

In [11]:
# Things related to "Barney Stinson"
queryString = """
SELECT DISTINCT ?p ?p_name ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q838512 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3938'), ('p_name', 'named by'), ('s', 'http://www.wikidata.org/entity/Q15793988'), ('name', 'cheerleader effect')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('p_name', 'spouse'), ('s', 'http://www.wikidata.org/entity/Q2347474'), ('name', 'Robin Scherbatsky')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('p_name', 'sibling'), ('s', 'http://www.wikidata.org/entity/Q50385667'), ('name', 'Carly Whittaker')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('p_name', 'sibling'), ('s', 'http://www.wikidata.org/entity/Q50383232'), ('name', 'James Stinson')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child'), ('s', 'http://www.wikidata.org/entity/Q50385684'), ('name', 'Jerome Whittaker')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child'), ('s', 'http://www.wikidata.org/entity/Q50383238'), ('name', 'Loretta Stinson')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('p_name', 

In [13]:
# "television series episode" details
queryString = """
SELECT DISTINCT ?p ?p_name ?s ?name
WHERE {
   # bind something
   wd:Q21191270 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template"), ('s', 'http://www.wikidata.org/entity/Q18606012'), ('name', 'Template:The Simpsons episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template"), ('s', 'http://www.wikidata.org/entity/Q7880352'), ('name', 'Template:Infobox television episode')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('s', 'http://www.wikidata.org/entity/Q39242358'), ('name', 'number of episodes produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('s', 'http://www.wikidata.org/entity/Q59180636'), ('name', 'episode title')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('s', 'http://www.wikidata.org/entity/Q39239591'), ('name', 'number of episodes at the end of the current season or of the last season aired')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 

In [14]:
# "television series episode" predicates
queryString = """
SELECT DISTINCT ?p ?p_name
WHERE {
   # bind something
   wd:Q21191270 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('p_name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('p_name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('p_name', 'maintained by WikiProject')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('p_name', "topic's main category")]
8


In [2]:
# istances of "television series episode"
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q21191270 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2343478'), ('name', "Move and You're Dead")]
[('s', 'http://www.wikidata.org/entity/Q8183269'), ('name', "A Hard Day's Night")]
[('s', 'http://www.wikidata.org/entity/Q8194509'), ('name', 'Danger at Ocean Deep')]
[('s', 'http://www.wikidata.org/entity/Q8773896'), ('name', 'Cry Wolf')]
[('s', 'http://www.wikidata.org/entity/Q9086804'), ('name', 'The First Cut Is the Deepest')]
[('s', 'http://www.wikidata.org/entity/Q13440704'), ('name', 'Bling')]
[('s', 'http://www.wikidata.org/entity/Q13505848'), ('name', 'Cut')]
[('s', 'http://www.wikidata.org/entity/Q13632733'), ('name', 'Fallacy')]
[('s', 'http://www.wikidata.org/entity/Q13638543'), ('name', 'Futility')]
[('s', 'http://www.wikidata.org/entity/Q13642560'), ('name', 'Girlfriends')]
[('s', 'http://www.wikidata.org/entity/Q13646055'), ('name', 'Good Faith')]
[('s', 'http://www.wikidata.org/entity/Q13647805'), ('name', 'Grief')]
[('s', 'http://www.wikidata.org/entity/Q13648798'), ('name', '

In [16]:
# "Television series episode" details
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q21191270 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18606012'), ('name', 'Template:The Simpsons episodes'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template")]
[('s', 'http://www.wikidata.org/entity/Q7880352'), ('name', 'Template:Infobox television episode'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template")]
[('s', 'http://www.wikidata.org/entity/Q39242358'), ('name', 'number of episodes produced'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('s', 'http://www.wikidata.org/entity/Q59180636'), ('name', 'episode title'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('s', 'http://www.wikidata.org/entity/Q39239591'), ('name', 'number of episodes at the end of the current season or of the last season aired'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('s', 'http://www.wikidata.org/entity/Q39242248'), ('name', 'nu

In [5]:
# "Annie Ilonzeh" details
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q465556 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('p_name', 'occupation')]
[('s', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('p_name', 'occupation')]
[('s', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('p_name', 'occupation')]
[('s', 'http://www.wikidata.org/entity/Q49085'), ('name', 'African Americans'), ('p', 'http://www.wikidata.org/prop/direct/P172'), ('p_name', 'ethnic group')]
[('s', 'http://www.wikidata.org/entity/Q185216'), ('name', 'Grapevine'), ('p', 'http://www.wikidata.org/prop/direct/P19'), ('p_name', 'place of birth')]
[('s', 'http://www.wikidata.org/entity/Q6581072'), ('name', 'female'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('p_name', 'sex or gender')]
[('s', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of 

In [8]:
# "HIMYM" details regarding episodes
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q147235 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
      
   FILTER regex(?name, "epis", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q785891'), ('name', 'list of How I Met Your Mother episodes'), ('p', 'http://www.wikidata.org/prop/direct/P1811'), ('p_name', 'list of episodes')]
[('s', 'http://www.wikidata.org/entity/Q21664088'), ('name', 'two-part episode'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('p_name', 'has parts of the class')]
2


In [9]:
# "list of How I Met Your Mother episodes" details
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q785891 ?p ?s .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8526738'), ('name', 'Category:How I Met Your Mother episodes'), ('p', 'http://www.wikidata.org/prop/direct/P1754'), ('p_name', 'category related to list')]
[('s', 'http://www.wikidata.org/entity/Q13406463'), ('name', 'Wikimedia list article'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q21191270'), ('name', 'television series episode'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('p_name', 'is a list of')]
3


In [10]:
# "Television series episode" predicates
queryString = """
SELECT DISTINCT ?p ?p_name
WHERE {
   # bind something
   wd:Q21191270 ?p ?s .
   # get the label
   ?p sc:name ?p_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('p_name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('p_name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('p_name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('p_name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('p_name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('p_name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('p_name', 'maintained by WikiProject')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('p_name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('p_name', 'KBpedi

In [11]:
# "Television series episode" is part of
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   wd:Q21191270 wdt:P361 ?s .
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('s', 'http://www.wikidata.org/entity/Q3464665'), ('name', 'television series season')]
2


In [14]:
# 'HIMYM' parts
queryString = """
SELECT DISTINCT ?obj ?o_name
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2438066'), ('o_name', 'How I Met Your Mother, season 6')]
[('obj', 'http://www.wikidata.org/entity/Q2715578'), ('o_name', 'How I Met Your Mother, season 1')]
[('obj', 'http://www.wikidata.org/entity/Q13567027'), ('o_name', 'How I Met Your Mother, season 9')]
[('obj', 'http://www.wikidata.org/entity/Q2567330'), ('o_name', 'How I Met Your Mother, season 4')]
[('obj', 'http://www.wikidata.org/entity/Q338715'), ('o_name', 'How I Met Your Mother, season 8')]
[('obj', 'http://www.wikidata.org/entity/Q582332'), ('o_name', 'How I Met Your Mother, season 5')]
[('obj', 'http://www.wikidata.org/entity/Q3468515'), ('o_name', 'How I Met Your Mother, season 2')]
[('obj', 'http://www.wikidata.org/entity/Q2555117'), ('o_name', 'How I Met Your Mother, season 3')]
[('obj', 'http://www.wikidata.org/entity/Q2472427'), ('o_name', 'How I Met Your Mother, season 7')]
9


In [19]:
# 'How I Met Your Mother, season 1' parts
queryString = """
SELECT DISTINCT ?obj ?o_name
WHERE {
   # bind something
   wd:Q2715578 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11696021'), ('o_name', 'Nothing Good Happens After 2 A.M.')]
[('obj', 'http://www.wikidata.org/entity/Q1327587'), ('o_name', 'Okay Awesome')]
[('obj', 'http://www.wikidata.org/entity/Q3480575'), ('o_name', 'Return of the Shirt')]
[('obj', 'http://www.wikidata.org/entity/Q467447'), ('o_name', 'Pilot')]
[('obj', 'http://www.wikidata.org/entity/Q468587'), ('o_name', 'Purple Giraffe')]
[('obj', 'http://www.wikidata.org/entity/Q471448'), ('o_name', 'Sweet Taste of Liberty')]
[('obj', 'http://www.wikidata.org/entity/Q4809956'), ('o_name', 'Game Night')]
[('obj', 'http://www.wikidata.org/entity/Q4817584'), ('o_name', 'Drumroll, Please')]
[('obj', 'http://www.wikidata.org/entity/Q4818636'), ('o_name', 'Life Among the Gorillas')]
[('obj', 'http://www.wikidata.org/entity/Q4818989'), ('o_name', 'Cupcake')]
[('obj', 'http://www.wikidata.org/entity/Q4819005'), ('o_name', 'The Wedding')]
[('obj', 'http://www.wikidata.org/entity/Q4884249'), ('o_name',

In [18]:
# 'Nothing Good Happens After 2 A.M.' cast
queryString = """
SELECT DISTINCT ?obj ?o_name
WHERE {
   # bind something
   wd:Q11696021 wdt:P161 ?obj .
   # get the label
   ?obj sc:name ?o_name.
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q200566'), ('o_name', 'Cobie Smulders')]
[('obj', 'http://www.wikidata.org/entity/Q435839'), ('o_name', 'Ashley Williams')]
[('obj', 'http://www.wikidata.org/entity/Q3045427'), ('o_name', 'George Cheung')]
[('obj', 'http://www.wikidata.org/entity/Q450496'), ('o_name', 'Alyssa Shafer')]
[('obj', 'http://www.wikidata.org/entity/Q485310'), ('o_name', 'Neil Patrick Harris')]
[('obj', 'http://www.wikidata.org/entity/Q223455'), ('o_name', 'Josh Radnor')]
[('obj', 'http://www.wikidata.org/entity/Q199927'), ('o_name', 'Alyson Hannigan')]
[('obj', 'http://www.wikidata.org/entity/Q202304'), ('o_name', 'Jason Segel')]
[('obj', 'http://www.wikidata.org/entity/Q333544'), ('o_name', 'Bob Saget')]
[('obj', 'http://www.wikidata.org/entity/Q366322'), ('o_name', 'Alexis Denisof')]
10


In [23]:
# Cast member and number of appearences
queryString = """
SELECT DISTINCT ?ac ?name (count(?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   # get the label
   ?ac sc:name ?name.
}
GROUP BY ?ac ?name
ORDER BY desc(?cnt)
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('ac', 'http://www.wikidata.org/entity/Q223455'), ('name', 'Josh Radnor'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q485310'), ('name', 'Neil Patrick Harris'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q200566'), ('name', 'Cobie Smulders'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q202304'), ('name', 'Jason Segel'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q199927'), ('name', 'Alyson Hannigan'), ('cnt', '143')]
[('ac', 'http://www.wikidata.org/entity/Q333544'), ('name', 'Bob Saget'), ('cnt', '142')]
[('ac', 'http://www.wikidata.org/entity/Q229914'), ('name', 'Lyndsy Fonseca'), ('cnt', '48')]
[('ac', 'http://www.wikidata.org/entity/Q297128'), ('name', 'David Henrie'), ('cnt', '48')]
[('ac', 'http://www.wikidata.org/entity/Q16149506'), ('name', 'Charlene Amoia'), ('cnt', '17')]
[('ac', 'http://www.wikidata.org/entity/Q199929'), ('name', 'Jennifer Morrison'), ('cnt', '12')]
[('ac', 'http://www.wikidata.org/entity/Q552889'

In [ ]:
# Total number of episodes
queryString = """
SELECT DISTINCT (count(?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   # get the label
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('cnt', '208')]
1


In [26]:
# Total number of episodes with at least 1 cast member
queryString = """
SELECT DISTINCT (count(distinct ?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   # get the label
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('cnt', '146')]
1


Final query for this task

In [27]:
# write your final query
# NOTE: The results does not match with the real-life data because on wikidata some episodes are missing the cast members
queryString = """
SELECT DISTINCT ?ac ?name (count(?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   # get the label
   ?ac sc:name ?name.
}
GROUP BY ?ac ?name
ORDER BY desc(?cnt)
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('ac', 'http://www.wikidata.org/entity/Q223455'), ('name', 'Josh Radnor'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q485310'), ('name', 'Neil Patrick Harris'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q200566'), ('name', 'Cobie Smulders'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q202304'), ('name', 'Jason Segel'), ('cnt', '145')]
[('ac', 'http://www.wikidata.org/entity/Q199927'), ('name', 'Alyson Hannigan'), ('cnt', '143')]
[('ac', 'http://www.wikidata.org/entity/Q333544'), ('name', 'Bob Saget'), ('cnt', '142')]
[('ac', 'http://www.wikidata.org/entity/Q229914'), ('name', 'Lyndsy Fonseca'), ('cnt', '48')]
[('ac', 'http://www.wikidata.org/entity/Q297128'), ('name', 'David Henrie'), ('cnt', '48')]
[('ac', 'http://www.wikidata.org/entity/Q16149506'), ('name', 'Charlene Amoia'), ('cnt', '17')]
[('ac', 'http://www.wikidata.org/entity/Q199929'), ('name', 'Jennifer Morrison'), ('cnt', '12')]
[('ac', 'http://www.wikidata.org/entity/Q552889'

## Task 4
Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

In [29]:
# Neil Patrick Harris predicates
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q485310 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1303'), ('name', 'instrument')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P412'), ('name', 'voice type')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'given name')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('name', 'residence')]
[('p', 'http://www.wikidata.org/prop/direct/P91'), ('name', 'sexual orientation')]
[('p', 'http://www.w

In [32]:
# things in which Neil Patrick Harris was a cast member
queryString = """
SELECT DISTINCT ?ep ?name
WHERE {
   # bind something
   ?ep wdt:P161 wd:Q485310 .
   OPTIONAL {
       ?ep wdt:P527 ?ss .
   } .
   OPTIONAL {
       ?ss wdt:P527 ?se .
   } .
   # get the label
   ?ep sc:name ?name .
   filter(?se != wd:Q147235)
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('ep', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother')]
[('ep', 'http://www.wikidata.org/entity/Q5264968'), ('name', 'Desperation Day')]
[('ep', 'http://www.wikidata.org/entity/Q5521981'), ('name', 'Garbage Island')]
[('ep', 'http://www.wikidata.org/entity/Q7080458'), ('name', 'Oh Honey')]
[('ep', 'http://www.wikidata.org/entity/Q5888848'), ('name', 'Home Wreckers')]
[('ep', 'http://www.wikidata.org/entity/Q7858076'), ('name', 'Twin Beds')]
[('ep', 'http://www.wikidata.org/entity/Q8074081'), ('name', 'Zoo or False')]
[('ep', 'http://www.wikidata.org/entity/Q1253366'), ('name', 'The Smurfs 2')]
[('ep', 'http://www.wikidata.org/entity/Q842253'), ('name', "Dr. Horrible's Sing-Along Blog")]
[('ep', 'http://www.wikidata.org/entity/Q2093198'), ('name', 'Undercover Brother')]
[('ep', 'http://www.wikidata.org/entity/Q4786988'), ('name', 'Architect of Destruction')]
[('ep', 'http://www.wikidata.org/entity/Q4838409'), ('name', 'Baby Talk')]
[('ep', 'http://w

In [33]:
# Home Wreckers predicates
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q5888848 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('name', 'screenwriter')]
[('p', 'http://www.wikidata.org/prop/direct/P4908'), ('name', 'season')]
9


In [34]:
# Home Wreckers is instance of
queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   wd:Q5888848 wdt:P31 ?o .
   # get the label
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q21191270'), ('o_name', 'television series episode')]
1


In [35]:
# things in which Neil Patrick Harris was a cast member != tv series episode
queryString = """
SELECT DISTINCT ?ep ?name
WHERE {
   # bind something
   ?ep wdt:P161 wd:Q485310 ;
       wdt:P31 ?o .
   # get the label
   ?ep sc:name ?name .
   filter(?o != wd:Q21191270)
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('ep', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother')]
[('ep', 'http://www.wikidata.org/entity/Q1253366'), ('name', 'The Smurfs 2')]
[('ep', 'http://www.wikidata.org/entity/Q842253'), ('name', "Dr. Horrible's Sing-Along Blog")]
[('ep', 'http://www.wikidata.org/entity/Q2093198'), ('name', 'Undercover Brother')]
[('ep', 'http://www.wikidata.org/entity/Q2779056'), ('name', 'Justice League: The New Frontier')]
[('ep', 'http://www.wikidata.org/entity/Q25340369'), ('name', 'A Series of Unfortunate Events')]
[('ep', 'http://www.wikidata.org/entity/Q607643'), ('name', 'Harold & Kumar Go to White Castle')]
[('ep', 'http://www.wikidata.org/entity/Q4764898'), ('name', 'Animal Room')]
[('ep', 'http://www.wikidata.org/entity/Q1165259'), ('name', 'Harold & Kumar Escape from Guantanamo Bay')]
[('ep', 'http://www.wikidata.org/entity/Q5659857'), ('name', 'Harold & Kumar')]
[('ep', 'http://www.wikidata.org/entity/Q16004491'), ('name', 'American Horror Story: Freak 

In [37]:
# Gone Girl is instance of
queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   wd:Q14920425 wdt:P31 ?o .
   # get the label
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('o_name', 'film')]
1


In [39]:
# films in which Neil Patrick Harris was a cast member
queryString = """
SELECT DISTINCT ?ep ?name
WHERE {
   # bind something
   ?ep wdt:P161 wd:Q485310 ;
       wdt:P31 ?o .
   # get the label
   ?ep sc:name ?name .
   filter(?o = wd:Q11424)
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('ep', 'http://www.wikidata.org/entity/Q2093198'), ('name', 'Undercover Brother')]
[('ep', 'http://www.wikidata.org/entity/Q2779056'), ('name', 'Justice League: The New Frontier')]
[('ep', 'http://www.wikidata.org/entity/Q607643'), ('name', 'Harold & Kumar Go to White Castle')]
[('ep', 'http://www.wikidata.org/entity/Q4764898'), ('name', 'Animal Room')]
[('ep', 'http://www.wikidata.org/entity/Q1165259'), ('name', 'Harold & Kumar Escape from Guantanamo Bay')]
[('ep', 'http://www.wikidata.org/entity/Q1247216'), ('name', 'The Next Best Thing')]
[('ep', 'http://www.wikidata.org/entity/Q478626'), ('name', 'Starship Troopers')]
[('ep', 'http://www.wikidata.org/entity/Q959335'), ('name', 'Cats & Dogs: The Revenge of Kitty Galore')]
[('ep', 'http://www.wikidata.org/entity/Q813057'), ('name', 'Beastly')]
[('ep', 'http://www.wikidata.org/entity/Q2384094'), ('name', 'Batman: Under the Red Hood')]
[('ep', 'http://www.wikidata.org/entity/Q550558'), ('name', 'The Muppets')]
[('ep', 'http://

In [4]:
# HIMYM start and end time
queryString = """
SELECT DISTINCT ?st ?et
WHERE {
   # bind something
   wd:Q147235 wdt:P580 ?st ;
              wdt:P582 ?et .
   # get the label
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00Z')]
1


In [7]:
# films in which Neil Patrick Harris was a cast member during HIMYM
queryString = """
SELECT DISTINCT ?ep ?name ?d ?st ?et
WHERE {
   # bind something
   ?ep wdt:P161 wd:Q485310 ;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?d .
    wd:Q147235 wdt:P580 ?st ;
               wdt:P582 ?et .
   # get the label
   ?ep sc:name ?name .
   filter(?d >= ?st && ?d <= ?et)
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('ep', 'http://www.wikidata.org/entity/Q2779056'), ('name', 'Justice League: The New Frontier'), ('d', '2008-01-01T00:00:00Z'), ('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00Z')]
[('ep', 'http://www.wikidata.org/entity/Q1165259'), ('name', 'Harold & Kumar Escape from Guantanamo Bay'), ('d', '2008-01-01T00:00:00Z'), ('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00Z')]
[('ep', 'http://www.wikidata.org/entity/Q1165259'), ('name', 'Harold & Kumar Escape from Guantanamo Bay'), ('d', '2008-03-23T00:00:00Z'), ('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00Z')]
[('ep', 'http://www.wikidata.org/entity/Q959335'), ('name', 'Cats & Dogs: The Revenge of Kitty Galore'), ('d', '2010-08-12T00:00:00Z'), ('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00Z')]
[('ep', 'http://www.wikidata.org/entity/Q959335'), ('name', 'Cats & Dogs: The Revenge of Kitty Galore'), ('d', '2010-01-01T00:00:00Z'), ('st', '2005-09-19T00:00:00Z'), ('et', '2014-03-31T00:00:00

Final query for this task

In [9]:
# write your final query

queryString = """
SELECT DISTINCT ?ac ?name (count(distinct ?ep_2) as ?num)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   ?ep_2 wdt:P161 ?ac ;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?d .
    wd:Q147235 wdt:P580 ?st ;
               wdt:P582 ?et .
   # get the label
   ?ac sc:name ?name .
   filter(?d >= ?st && ?d <= ?et)
}
ORDER BY desc(?num)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('ac', 'http://www.wikidata.org/entity/Q192165'), ('name', 'Danny Glover'), ('num', '40')]
[('ac', 'http://www.wikidata.org/entity/Q229669'), ('name', 'Malin Åkerman'), ('num', '21')]
[('ac', 'http://www.wikidata.org/entity/Q236189'), ('name', 'Judy Greer'), ('num', '18')]
[('ac', 'http://www.wikidata.org/entity/Q233433'), ('name', 'Rachelle Lefevre'), ('num', '17')]
[('ac', 'http://www.wikidata.org/entity/Q1319744'), ('name', 'Will Forte'), ('num', '17')]
[('ac', 'http://www.wikidata.org/entity/Q1189470'), ('name', 'Jimmi Simpson'), ('num', '16')]
[('ac', 'http://www.wikidata.org/entity/Q566037'), ('name', 'Scoot McNairy'), ('num', '16')]
[('ac', 'http://www.wikidata.org/entity/Q530646'), ('name', 'Ray Wise'), ('num', '16')]
[('ac', 'http://www.wikidata.org/entity/Q23547'), ('name', 'Bryan Cranston'), ('num', '14')]
[('ac', 'http://www.wikidata.org/entity/Q272927'), ('name', 'Kristen Schaal'), ('num', '14')]
[('ac', 'http://www.wikidata.org/entity/Q312705'), ('name', 'John Ch

## Task 5
Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [40]:
# Total number of seasons and episodes of HIMYM
queryString = """
SELECT DISTINCT (count(distinct ?se) as ?se_cnt) (count(?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   # get the label
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('se_cnt', '9'), ('cnt', '208')]
1


In [42]:
# Total number of seasons, episodes and cast members of HIMYM and The Office
queryString = """
SELECT DISTINCT ?ts ?name (count(distinct ?se) as ?se_cnt) (count(distinct ?ep) as ?ep_cnt) (count(distinct ?ac) as ?ac_cnt)
WHERE {
   # bind something
   ?ts wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   # get the label
   ?ts sc:name ?name.
   
   FILTER (?ts in (wd:Q147235, wd:Q23831))
}
GROUP BY ?ts ?name
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('ts', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother'), ('se_cnt', '9'), ('ep_cnt', '146'), ('ac_cnt', '214')]
[('ts', 'http://www.wikidata.org/entity/Q23831'), ('name', 'The Office'), ('se_cnt', '1'), ('ep_cnt', '1'), ('ac_cnt', '7')]
2


In [43]:
# Total number of seasons and episodes of The Office
queryString = """
SELECT DISTINCT (count(distinct ?se) as ?se_cnt) (count(?ep) as ?cnt)
WHERE {
   # bind something
   wd:Q23831 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   # get the label
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('se_cnt', '9'), ('cnt', '201')]
1


In [45]:
# Total number of seasons, episodes and cast members of HIMYM and The Office
queryString = """
SELECT DISTINCT ?ts ?name (count(distinct ?se) as ?se_cnt) (count(distinct ?ep) as ?ep_cnt) (count(distinct ?ac) as ?ac_cnt)
WHERE {
   # bind something
   ?ts wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   OPTIONAL {
       ?ep wdt:P161 ?ac .
   }
   # get the label
   ?ts sc:name ?name.
   
   FILTER (?ts=wd:Q147235 || ?ts=wd:Q23831)
}
GROUP BY ?ts ?name
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('ts', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother'), ('se_cnt', '9'), ('ep_cnt', '208'), ('ac_cnt', '214')]
[('ts', 'http://www.wikidata.org/entity/Q23831'), ('name', 'The Office'), ('se_cnt', '9'), ('ep_cnt', '201'), ('ac_cnt', '7')]
2


Final query for this task

In [46]:
# write your final query
# NOTE: The cast member count for "The Office" is clearly wrong because the property is missing on almost all episodes
queryString = """
SELECT DISTINCT ?ts ?name (count(distinct ?se) as ?se_cnt) (count(distinct ?ep) as ?ep_cnt) (count(distinct ?ac) as ?ac_cnt)
WHERE {
   # bind something
   ?ts wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   OPTIONAL {
       ?ep wdt:P161 ?ac .
   }
   # get the label
   ?ts sc:name ?name.
   
   FILTER (?ts=wd:Q147235 || ?ts=wd:Q23831)
}
GROUP BY ?ts ?name
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('ts', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother'), ('se_cnt', '9'), ('ep_cnt', '208'), ('ac_cnt', '214')]
[('ts', 'http://www.wikidata.org/entity/Q23831'), ('name', 'The Office'), ('se_cnt', '9'), ('ep_cnt', '201'), ('ac_cnt', '7')]
2


## Task 6
Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [11]:
# Subjects with profession actor named Kevin Bacon
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P106 wd:Q33999 .
   # get the label
   ?s sc:name ?name.
   FILTER regex(?name, "Kevin Bacon", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3454165'), ('name', 'Kevin Bacon')]
1


In [40]:
# Actors with Kevin Bacon number = 2
queryString = """
SELECT DISTINCT ?cast_2 ?name (min(?bn) as ?b)
WHERE {
   # bind something
   ?kbf_0 wdt:P161 wd:Q3454165 ;
        wdt:P161 ?cast_1 .
   ?kbf_1 wdt:P161 ?cast_1 ;
        wdt:P161 ?cast_2 .
    BIND (
      IF(?cast_2 = wd:Q3454165, 0,
        IF(?cast_2 in (?cast_1), 1, 2)
      ) AS ?bn 
    ) .
   # get the label
   ?cast_2 sc:name ?name.
   FILTER regex(?name, "Arnold S", "i")
   
}
GROUP BY ?cast_2 ?name
HAVING (MIN(?bn) = 2)
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('cast_2', 'http://www.wikidata.org/entity/Q2685'), ('name', 'Arnold Schwarzenegger'), ('b', '2')]
[('cast_2', 'http://www.wikidata.org/entity/Q2348194'), ('name', 'Arnold Stang'), ('b', '2')]
2


In [41]:
# WRONG QUERY
queryString = """
SELECT DISTINCT ?cast_2 ?name ?bn
WHERE {
   # bind something
   ?kbf_0 wdt:P161 wd:Q3454165 ;
        wdt:P161 ?cast_1 .
   ?kbf_1 wdt:P161 ?cast_1 ;
        wdt:P161 ?cast_2 .
    filter exists {?cast_2 a (?cast_1)
    }
   # get the label
   ?cast_2 sc:name ?name.
   FILTER regex(?name, "Andie MacDowell", "i")
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [42]:
# write your final query
queryString = """
SELECT DISTINCT ?cast_2 ?name
WHERE {
   # bind something
   ?kbf_0 wdt:P161 wd:Q3454165 ;
        wdt:P161 ?cast_1 .
   ?kbf_1 wdt:P161 ?cast_1 ;
        wdt:P161 ?cast_2 .
    BIND (
      IF(?cast_2 = wd:Q3454165, 0,
        IF(?cast_2 in (?cast_1), 1, 2)
      ) AS ?bn 
    ) .
   # get the label
   ?cast_2 sc:name ?name.   
}
GROUP BY ?cast_2 ?name
HAVING (MIN(?bn) = 2)
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('cast_2', 'http://www.wikidata.org/entity/Q1280116'), ('name', 'Rachael Crawford')]
[('cast_2', 'http://www.wikidata.org/entity/Q5292601'), ('name', 'Don Fernando')]
[('cast_2', 'http://www.wikidata.org/entity/Q24961979'), ('name', 'Miguel Ángel Porro')]
[('cast_2', 'http://www.wikidata.org/entity/Q26830465'), ('name', 'David H. Holmes')]
[('cast_2', 'http://www.wikidata.org/entity/Q15439890'), ('name', 'Jonathan Pienaar')]
[('cast_2', 'http://www.wikidata.org/entity/Q18602205'), ('name', 'Slaven Španović')]
[('cast_2', 'http://www.wikidata.org/entity/Q27898516'), ('name', 'Katie West')]
[('cast_2', 'http://www.wikidata.org/entity/Q7295103'), ('name', 'Rasmus Hardiker')]
[('cast_2', 'http://www.wikidata.org/entity/Q192955'), ('name', 'Steve Nash')]
[('cast_2', 'http://www.wikidata.org/entity/Q3183213'), ('name', 'Jon Edward Martin')]
[('cast_2', 'http://www.wikidata.org/entity/Q106072094'), ('name', 'Napoleon Highbrou')]
[('cast_2', 'http://www.wikidata.org/entity/Q27733240')

## Task 7
Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [47]:
# Predicates of HIMYM regarding award
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
   FILTER regex(?name, "award", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
1


In [8]:
# Awards reveived by HIMYM
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q147235 wdt:P166 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2300462'), ('name', 'Streamy Awards')]
[('o', 'http://www.wikidata.org/entity/Q3002950'), ('name', "Critics' Choice Television Award for Best Supporting Actor in a Comedy Series")]
[('o', 'http://www.wikidata.org/entity/Q7243510'), ('name', 'Primetime Emmy Award for Outstanding Single-Camera Picture Editing for a Comedy Series')]
[('o', 'http://www.wikidata.org/entity/Q23011211'), ('name', 'International TV Audience Award for Best Comedy TV Series')]
[('o', 'http://www.wikidata.org/entity/Q7243501'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Multi-Camera Series')]
[('o', 'http://www.wikidata.org/entity/Q17985609'), ('name', 'Primetime Emmy Award for Outstanding Art Direction for a Contemporary Program')]
[('o', 'http://www.wikidata.org/entity/Q30633707'), ('name', 'Primetime Emmy Award for Outstanding Multi-Camera Picture Editing for a Comedy Series')]
7


In [4]:
# Awards won by series in which actors of HIMYM acted
queryString = """
SELECT DISTINCT ?ts ?name (count(?aw) as ?cnt_aw)
WHERE {
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   ?ac wdt:P161 ?ep_2 .
   ?ep_2 wdt:P527 ?se_2 .
   ?se_2 wdt:P527 ?ts .
   ?ts wdt:P166 ?aw .
   # get the label
   ?ts sc:name ?name.
}
GROUP BY ?ts ?name
ORDER BY desc(?cnt_aw)
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
# Awards won by series in which actors of HIMYM acted
queryString = """
SELECT DISTINCT ?ts ?name (count(distinct ?aw) as ?cnt_aw)
WHERE {
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   ?ep_2 wdt:P161 ?ac .
   ?se_2 wdt:P527 ?ep_2 .
   ?ts wdt:P527 ?se_2 .
   ?ts wdt:P166 ?aw .
   # get the label
   ?ts sc:name ?name.
}
GROUP BY ?ts ?name
ORDER BY desc(?cnt_aw)
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('ts', 'http://www.wikidata.org/entity/Q3577037'), ('name', 'The West Wing'), ('cnt_aw', '22')]
[('ts', 'http://www.wikidata.org/entity/Q1079'), ('name', 'Breaking Bad'), ('cnt_aw', '21')]
[('ts', 'http://www.wikidata.org/entity/Q79784'), ('name', 'Friends'), ('cnt_aw', '10')]
[('ts', 'http://www.wikidata.org/entity/Q8539'), ('name', 'The Big Bang Theory'), ('cnt_aw', '10')]
[('ts', 'http://www.wikidata.org/entity/Q438406'), ('name', "Grey's Anatomy"), ('cnt_aw', '8')]
[('ts', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother'), ('cnt_aw', '7')]
[('ts', 'http://www.wikidata.org/entity/Q558112'), ('name', 'Black Mirror'), ('cnt_aw', '6')]
[('ts', 'http://www.wikidata.org/entity/Q2085'), ('name', 'Twin Peaks'), ('cnt_aw', '6')]
[('ts', 'http://www.wikidata.org/entity/Q13979'), ('name', 'Saturday Night Live'), ('cnt_aw', '2')]
[('ts', 'http://www.wikidata.org/entity/Q2744'), ('name', 'The X-Files'), ('cnt_aw', '1')]
[('ts', 'http://www.wikidata.org/entity

Final query for this task

In [5]:
# write your final query

queryString = """
SELECT DISTINCT ?ts ?name (count(distinct ?aw) as ?cnt_aw)
WHERE {
   wd:Q147235 wdt:P527 ?se .
   ?se wdt:P527 ?ep .
   ?ep wdt:P161 ?ac .
   ?ep_2 wdt:P161 ?ac .
   ?se_2 wdt:P527 ?ep_2 .
   ?ts wdt:P527 ?se_2 .
   ?ts wdt:P166 ?aw .
   # get the label
   ?ts sc:name ?name.
}
GROUP BY ?ts ?name
HAVING (count(distinct ?aw) > 2)
ORDER BY desc(?cnt_aw)
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('ts', 'http://www.wikidata.org/entity/Q3577037'), ('name', 'The West Wing'), ('cnt_aw', '22')]
[('ts', 'http://www.wikidata.org/entity/Q1079'), ('name', 'Breaking Bad'), ('cnt_aw', '21')]
[('ts', 'http://www.wikidata.org/entity/Q79784'), ('name', 'Friends'), ('cnt_aw', '10')]
[('ts', 'http://www.wikidata.org/entity/Q8539'), ('name', 'The Big Bang Theory'), ('cnt_aw', '10')]
[('ts', 'http://www.wikidata.org/entity/Q438406'), ('name', "Grey's Anatomy"), ('cnt_aw', '8')]
[('ts', 'http://www.wikidata.org/entity/Q147235'), ('name', 'How I Met Your Mother'), ('cnt_aw', '7')]
[('ts', 'http://www.wikidata.org/entity/Q558112'), ('name', 'Black Mirror'), ('cnt_aw', '6')]
[('ts', 'http://www.wikidata.org/entity/Q2085'), ('name', 'Twin Peaks'), ('cnt_aw', '6')]
8
